In [54]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *
from losses import *
from plots import *
from EDA import *
from cross_validation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of cross_validation failed: Traceback (most recent call last):
  File "/Users/gaspardvilla/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/gaspardvilla/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/gaspardvilla/opt/anaconda3/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/gaspardvilla/opt/anaconda3/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/gaspardvilla/Documents/EPFL/5ème année/Semestre IX/Machine Learning/Workspace/Project 1/scripts/cross_validation.

## Load the training data into feature matrix, class labels, and event ids:

In [55]:
DATA_TRAIN_PATH = "../data/train.csv"
data_y, data_set, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

In [56]:
# Fixing the seed
seed = 8

### Split and classify data

In [57]:
# Classification of the output
y_0, y_1, y_2, y_3 = y_classification(data_y, data_set)

In [58]:
#EDA for each class
class_0, class_1, class_2, class_3 = EDA_class(data_set)

In [59]:
# Set the "spliter" value correponding to the part dedicated for train
spliter = 0.8

# Split into a train and a test set
train_0, y_tr_0, test_0, y_te_0 = train_test_separator(y_0, class_0)
train_1, y_tr_1, test_1, y_te_1 = train_test_separator(y_1, class_1)
train_2, y_tr_2, test_2, y_te_2 = train_test_separator(y_2, class_2)
train_3, y_tr_3, test_3, y_te_3 = train_test_separator(y_3, class_3)

# Test for logistic and reguralized logitic regressions

***

In [ ]:
# Setting parameters
neg_log = Neg_log()
param_log = Parameters()
param_log.set_degree(10)
param_log.set_gamma(1e-6)
param_log.set_lambda(1e-2)
param_log.set_max_iter(100)
param_log.set_method(ridge_regression)
param_log.set_loss_fct(neg_log)
param_log.set_to_test(['lambda'])
param_log.set_viz(False)
param_log.set_use_backward_selection(False)
param_log.set_use_interactions(True)

In [ ]:
loss, w = ridge_regression(y_0, class_0, param_log)
y_pred = predict_labels(w, class_0)

In [ ]:
counting_errors(y_pred, y_0)

# Degree test

***

In [60]:
param = Parameters()
param.set_degree(6)
param.set_method(ridge_regression)
param.set_to_test(['lambda'])
param.set_viz(False)
param.set_use_backward_selection(False)
param.set_use_interactions(True)

In [61]:
param = cross_validation_poly_gas(y_0, class_0, param)
param.polynomial_selection

yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
yolo
forward:  14.811083123425693
backward:  100


'Forward'

In [62]:
print(param.feature_list)
print(param.best_error)
print(class_0.shape)
print(param.kept_interactions)

for i in [1,2]:
    print(i)

[[0. 0. 3. 4. 8. 1. 3. 6. 7. 8. 9. 2. 4. 7. 1. 3. 1. 2. 8. 0.]
 [0. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 3. 3. 3. 4. 4. 5. 5. 5. 6.]]
14.811083123425693
(1985, 10)
[[0. 1. 3. 3.]
 [0. 2. 6. 7.]]
1
2


In [ ]:
print(param.feature_list)
print(param.best_error)
print(class_0.shape)

for i in [1,2]:
    print(i)

# Tutrial for cross validation

***

In [ ]:
# Create a "paramters" object
param = Parameters()

# Set up some parameters such as 'max_iter', 'gamma', etc...
param.set_max_iter(100)
param.set_mini_batch_size(100)
param.set_gamma(1e-1) # useless for the cross-validation but it's an example

# Define the method and the loss function you want
param.set_method(least_squares_GD)
mse = MSE()
param.set_loss_fct(mse)

# Select the parameter you want to test ('gamma' or 'lambda' or both)
param.set_to_test(['gamma'])

# Define the range of your parameter that will be test
param.set_gamma_range(np.logspace(-10,0,30))

# Tell if you want visualization of your result or not
param.set_viz(True)

# Select the class of your data set: 0, 1, 2 or 3
class_ind = 0

In [ ]:
for idx in range(4):
    opt_param = test_function(data_y, data_set, param, idx)

# See some histograms

***

#### Only on class 0

In [ ]:
classs = class_0
yy = y_0

In [ ]:
ind_y_neg = np.array(np.where(yy == 0)[0])
ind_y_pos = np.array(np.where(yy == 1)[0])

In [ ]:
for col in range(16):
    plt.figure(figsize=(8,5))
    plt.hist(classs[ind_y_neg, col], bins=100, histtype = 'step', color = 'red', density=True)
    plt.hist(classs[ind_y_pos, col], bins=100, histtype = 'step', density=True)
    plt.title('Histogram of feature 0')
    plt.show()

***
***

# Submission example

In [ ]:
param = Parameters()
param.set_method(ridge_regression)
param.set_lambda(1e-3)
param.set_method(ridge_regression)
degree = 10

In [ ]:
# Train our model and get the response weight and its loss
param.set_init_w(np.zeros(build_poly(class_0, degree).shape[1]))
mse_0, w_0 = param.method(y_0, build_poly(class_0, degree) , param)

In [ ]:
# Train our model and get the response weight and its loss
param.set_init_w(np.zeros(build_poly(class_1, degree).shape[1]))
mse_1, w_1 = param.method(y_1, build_poly(class_1, degree) , param)

In [ ]:
# Train our model and get the response weight and its loss
param.set_init_w(np.zeros(build_poly(class_2, degree).shape[1]))
mse_2, w_2 = param.method(y_2, build_poly(class_2, degree) , param)

In [ ]:
# Train our model and get the response weight and its loss
param.set_init_w(np.zeros(build_poly(class_3, degree).shape[1]))
mse_3, w_3 = param.method(y_3, build_poly(class_3, degree) , param)

In [ ]:
print(w_0.shape)
print(class_0.shape)

In [ ]:
# Get the predictions on the train set
y_pred_0 = predict_labels(w_0, build_poly(class_0, degree))
y_pred_1 = predict_labels(w_1, build_poly(class_1, degree))
y_pred_2 = predict_labels(w_2, build_poly(class_2, degree))
y_pred_3 = predict_labels(w_3, build_poly(class_3, degree))

In [ ]:
y_pred_train = rebuild_y(y_pred_0, y_pred_1, y_pred_2, y_pred_3, data_set)
counting_errors(y_pred_train, data_y)

In [ ]:
DATA_TEST_PATH = "../data/test.csv"
_, data_test_set, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
# EDA for each class
class_0, class_1, class_2, class_3 = EDA_class(data_test_set)

# Remove from graoh analysis
class_0 = remove_feature(class_0, [2, 5, 8, 11, 13])
class_1 = remove_feature(class_1, [4, 10, 11, 13, 14, 15, 18])
class_2 = remove_feature(class_2, [2, 5, 8, 11, 13])
class_3 = remove_feature(class_3, [5, 7, 10, 13, 14, 16, 17, 18, 19, 20, 22, 23, 25, 26])

# Get the predictions on the train set
y_pred_0 = predict_labels(w_0, build_poly(class_0, degree))
y_pred_1 = predict_labels(w_1, build_poly(class_1, degree))
y_pred_2 = predict_labels(w_2, build_poly(class_2, degree))
y_pred_3 = predict_labels(w_3, build_poly(class_3, degree))

y_pred = rebuild_y(y_pred_0, y_pred_1, y_pred_2, y_pred_3, data_test_set)

In [ ]:
OUTPUT_PATH = '../data/test_prediction_submission.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)